### Import pygooglenews

In [3]:
from pygooglenews import GoogleNews
import pandas as pd
import numpy as np

gn = GoogleNews() # now global mode? can specify country = US

### Import .csv file 

In [6]:
df = pd.read_csv('UseCase1_Data.csv') # must be in same repository that the jupyter notebook is in

In [7]:
df

,Author_Name,Article_Title,Article_DOI
0,Laurel Westbrook,"Doing Gender, Determining Gender: Transgender ...",10.1177/0891243213503203
1,Jonathan Ospina-Betancurt,The End of Compulsory Gender Verification: Is ...,10.1007/s10508-021-02073-x
2,Katrina Karkazis,The misuses of “biological sex”,10.1016/S0140-6736(19)32764-3
3,Vanessa Heggie,"Testing sex and gender in sports; reinventing,...",10.1016/j.endeavour.2010.09.005
4,April Vannini,"Girl, Interrupted: Interpreting Semenya’s Body...",10.1177/1532708611409536
5,Heather Sykes,Transsexual and Transgender Policies in Sport,10.1123/wspaj.15.1.3
6,Stephane Bermon,Are the New Policies on Hyperandrogenism in El...,10.1080/15265161.2013.776129
7,Francisco Sanchez,The New Policy on Hyperandrogenism in Elite Fe...,10.1080/00224499.2012.752429
8,Ruth Wood,Testosterone and sport: current perspectives,10.1016/j.yhbeh.2011.09.010
9,Emma Hilton,Transgender women in the female category of sp...,10.1007/s40279-020-01389-3


### Get common key words in title first

In [36]:
article_titles = df['Article_Title'].to_list() # make article titles a list
print('article_titles: ', article_titles)
print(type(article_titles))


article_titles:  ['Doing Gender, Determining Gender:\xa0Transgender People, Gender Panics, and the Maintenance of the Sex/Gender/Sexuality System', 'The End of Compulsory Gender Verification: Is It Progress for Inclusion of Women in Sports?', 'The misuses of “biological sex”', 'Testing sex and gender in sports; reinventing, reimagining and reconstructing histories', 'Girl, Interrupted: Interpreting Semenya’s Body, Gender Verification Testing, and Public Discourse', 'Transsexual and Transgender Policies in Sport', 'Are the New Policies on Hyperandrogenism in Elite Female Athletes Really Out of Bounds? Response to “Out of Bounds? A Critique of the New Policies on Hyperandrogenism in Elite Female Athletes”', 'The New Policy on Hyperandrogenism in Elite Female Athletes is Not About “Sex Testing”', 'Testosterone and sport: current perspectives', 'Transgender women in the female category of sport: perspectives on testosterone suppression and performance advantage', 'Implications of a third g

In [44]:
from collections import Counter

stops = {'this', 'that', 'a', 'is', "and", "Determining", "Doing", "the", "People", "of"}    
words = article_titles[0].split()
count = Counter(word for word in words if word not in stops).most_common(10) # we may want to figure out a way to remove all punctuation from titles first

print(count)



[('Gender,', 1), ('Gender:', 1), ('Transgender', 1), ('People,', 1), ('Gender', 1), ('Panics,', 1), ('Maintenance', 1), ('Sex/Gender/Sexuality', 1), ('System', 1)]


NameError: name 'to' is not defined

### Function for scraping 

In [ ]:
# search = gn.search(‘author name’) #search by author name
res_df = pd.DataFrame()
def scraping(df):
    stories = []
    for i in range(len(df)):
        search = gn.search(df.iloc[i].loc['Author_Name'])
        newsitem = search['entries']
        for item in newsitem:
            story = {
                # authorname
                'title': item.title,
                'link': item.link
            }
            stories.append(story)
    return stories
scraping(df)